In [1]:
import cv2
import tensorflow as tf
import argparse
import numpy as np
import os
import pdb
import time
import matplotlib.pyplot as plt
import sys
print(tf.__version__)
import scipy.io as sio
import math

############### Global Parameters ###############
# path
video_list  = sio.loadmat('trainlist1.mat')
test_list = sio.loadmat('testlist1.mat')

train_path = './experiments/hmdb-51/rgb_data_split1/train64/'

train_path_2 = './experiments/hmdb-51/flow_data_split1/train64/'

test_path = './experiments/hmdb-51/rgb_data_split1/test64/'

test_path_2 = './experiments/hmdb-51/flow_data_split1/test64/'

train_path_3 = './experiments/hmdb-51/flow_data_split1/train64_traabu/'

test_path_3 = './experiments/hmdb-51/flow_data_split1/test64_traabu/'

# Model Path
default_model_path = './multi_model/multi_att_1_fusionft_3s/final_model'
save_path = './multi_model/multi_att_1_fusionft_3s/'

# Output Path
output_path = './multi_att_1_fusionft_3s_res/'

# Train Test Split
train_num = video_list['video'][0].shape[0]-1
test_num = test_list['video'][0].shape[0]-1

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
tf.test.is_gpu_available()

/usr/local/lib/python2.7/dist-packages/requests/__init__.py:83: RequestsDependencyWarning: Old version of cryptography ([1, 2, 3]) may cause slowdown.
  warnings.warn(warning, RequestsDependencyWarning)


1.10.0


True

In [2]:
video_list ['video'][0][1][0]

array([u'April_09_brush_hair_u_nm_np1_ba_goo_1'], dtype='<U37')

In [3]:
############## Train Parameters #################
# Parameters
learning_rate = 0.00001
n_epochs = 150
batch_size = 1
# Network Parameters
n_input = 1024 # input dimension
n_hidden = 1024 # hidden layer num of multi_head
n_classes = 51 # has 51 classes
n_frames = 64

In [4]:
def squared_dist(A, B):
    assert A.shape.as_list() == B.shape.as_list()

    row_norms_A = tf.reduce_sum(tf.square(A), axis=1)
    row_norms_A = tf.reshape(row_norms_A, [-1, 1])  # Column vector.

    row_norms_B = tf.reduce_sum(tf.square(B), axis=1)
    row_norms_B = tf.reshape(row_norms_B, [1, -1])  # Row vector.

    return row_norms_A - 2 * tf.matmul(A, tf.transpose(B)) + row_norms_B

In [5]:
def normalize(inputs, 
              epsilon = 1e-8,
              scope="ln",
              reuse=None):
    '''Applies layer normalization.
    
    Args:
      inputs: A tensor with 2 or more dimensions, where the first dimension has
        `batch_size`.
      epsilon: A floating number. A very small number for preventing ZeroDivision Error.
      scope: Optional scope for `variable_scope`.
      reuse: Boolean, whether to reuse the weights of a previous layer
        by the same name.
      
    Returns:
      A tensor with the same shape and data dtype as `inputs`.
    '''
    with tf.variable_scope(scope, reuse=reuse):
        inputs_shape = inputs.get_shape()
        params_shape = inputs_shape[-1:]
    
        mean, variance = tf.nn.moments(inputs, [-1], keep_dims=True)
        beta= tf.Variable(tf.zeros(params_shape))
        gamma = tf.Variable(tf.ones(params_shape))
        normalized = (inputs - mean) / ( (variance + epsilon) ** (.5) )
        outputs = gamma * normalized + beta
        
    return outputs

In [6]:
def multihead_attention(queries, 
                        keys,
                        wkeys, 
                        num_units=None, 
                        num_heads=8, 
                        dropout_rate=0,
                        is_training=True,
                        causality=False,
                        trainable=True,
                        scope="multihead_attention", 
                        reuse=None):
    '''Applies multihead attention.
    
    Args:
      queries: A 3d tensor with shape of [N, T_q, C_q].
      keys: A 3d tensor with shape of [N, T_k, C_k].
      num_units: A scalar. Attention size.
      dropout_rate: A floating point number.
      is_training: Boolean. Controller of mechanism for dropout.
      causality: Boolean. If true, units that reference the future are masked. 
      num_heads: An int. Number of heads.
      scope: Optional scope for `variable_scope`.
      reuse: Boolean, whether to reuse the weights of a previous layer
        by the same name.
        
    Returns
      A 3d tensor with shape of (N, T_q, C)  
    '''
    with tf.variable_scope(scope, reuse=reuse):
        # Set the fall back option for num_units
        if num_units is None:
            num_units = queries.get_shape().as_list[-1]
        
        # Linear projections
        Q = tf.contrib.layers.fully_connected(queries, num_units,trainable=trainable) # (N, T_q, C)
        K = tf.contrib.layers.fully_connected(queries, num_units,trainable=trainable ) # (N, T_k, C)
        V = tf.contrib.layers.fully_connected(queries, num_units,trainable=trainable) # (N, T_k, C)
        
        Q1 = tf.reshape(Q,(1,n_frames,num_units))
        K1 = tf.reshape(K,(1,n_frames,num_units))        
        V1 = tf.reshape(V,(1,n_frames,num_units))
        
        KF = tf.contrib.layers.fully_connected(keys, num_units,trainable=trainable)
        KF_= tf.concat(tf.split(KF, num_heads, axis = 2),axis =0) # (h*N, T_k, C/h) 
        QF = tf.contrib.layers.fully_connected(keys, num_units,trainable=trainable)
        QF_= tf.concat(tf.split(QF, num_heads, axis = 2),axis =0) # (h*N, T_k, C/h)
        
        # Split and concat
        Q_ = tf.concat(tf.split(Q1, num_heads, axis = 2),axis =0) # (h*N, T_q, C/h) 
        K_ = tf.concat(tf.split(K1, num_heads, axis = 2),axis =0) # (h*N, T_k, C/h) 
        V_ = tf.concat(tf.split(V1, num_heads, axis = 2),axis =0) # (h*N, T_k, C/h) 

        
        # Multiplication
        outputs  = tf.matmul(Q_, tf.transpose(K_, [0, 2, 1])) # (h*N, T_q, T_k)
        output_1 = tf.matmul(QF_, wkeys) # (h*N, T_q, T_k)
        output_2 = tf.matmul(output_1, tf.transpose(KF_, [0, 2, 1])) # (h*N, T_q, T_k)
        
        outputs  = tf.multiply( outputs + output_2,0.5)
        
        
        # Scale
        outputs = outputs / (K_.get_shape().as_list()[-1] ** 0.5)

                
        # Key Masking
        key_masks = tf.sign(tf.abs(tf.reduce_sum(keys, axis=-1))) # (N, T_k)
        key_masks = tf.tile(key_masks, [num_heads, 1]) # (h*N, T_k)
        key_masks = tf.tile(tf.expand_dims(key_masks, 1), [1, tf.shape(queries)[1], 1]) # (h*N, T_q, T_k)
       # 
        paddings = tf.ones_like(outputs)*(-2**32+1)
        outputs = tf.where(tf.equal(key_masks, 0), paddings, outputs) # (h*N, T_q, T_k)
          
        # Causality = Future blinding
        if causality:
            diag_vals = tf.ones_like(outputs[0, :, :]) # (T_q, T_k)
            tril = tf.contrib.linalg.LinearOperatorTriL(diag_vals).to_dense() # (T_q, T_k)
            masks = tf.tile(tf.expand_dims(tril, 0), [tf.shape(outputs)[0], 1, 1]) # (h*N, T_q, T_k)
  # 
            paddings = tf.ones_like(masks)*(-2**32+1)
            outputs = tf.where(tf.equal(masks, 0), paddings, outputs) # (h*N, T_q, T_k)
  
        # Activation
        outputs = tf.nn.softmax(outputs) # (h*N, T_q, T_k)
        matt    = outputs
        
        
        # Query Masking
        query_masks = tf.sign(tf.abs(tf.reduce_sum(queries, axis=-1))) # (N, T_q)
        query_masks = tf.tile(query_masks, [num_heads, 1]) # (h*N, T_q)
        query_masks = tf.tile(tf.expand_dims(query_masks, -1), [1, 1, tf.shape(keys)[1]]) # (h*N, T_q, T_k)
        outputs *= query_masks # broadcasting. (N, T_q, C)
          
        # Dropouts
        outputs = tf.contrib.layers.dropout(outputs, keep_prob=dropout_rate, is_training=tf.convert_to_tensor(is_training))
               
        # Weighted sum
        outputs = tf.matmul(outputs, V_) # ( h*N, T_q, C/h)
        
        # Restore shape
        outputs = tf.concat(tf.split(outputs, num_heads,axis = 0),axis =2 ) # (N, T_q, C)
              
        # Residual connection
        outputs += queries
              
        # Normalize
        outputs = normalize(outputs) # (N, T_q, C)
 
    return outputs, matt

In [7]:
def build_model():

    # tf Graph input
    x   = tf.placeholder("float", [None, n_frames , n_input])
    x2  = tf.placeholder("float", [None, n_frames , n_input])
    x3  = tf.placeholder("float", [None, n_frames , n_input])
    y   = tf.placeholder("float", [None, n_classes])
    # Define weights
    weights = {
        'out'    : tf.Variable(tf.random_normal([n_hidden, n_classes], mean=0.0, stddev=0.01)),
        'att_wk' : tf.Variable(tf.random_normal([8,n_input/8, n_input/8], mean=0.0, stddev=0.02),trainable=True),
        'att_wk2': tf.Variable(tf.random_normal([8,n_input/8, n_input/8], mean=0.0, stddev=0.02),trainable=True),
        'att_wk3': tf.Variable(tf.random_normal([8,n_input/8, n_input/8], mean=0.0, stddev=0.02),trainable=True)

    }
    biases = {
        'out': tf.Variable(tf.random_normal([n_classes], mean=0.0, stddev=0.01))


    }
    # Attention Weighting
#     m1 = tf.get_variable('m1',dtype=tf.float32, shape = [1])
#     m2 = tf.get_variable('m2',dtype=tf.float32, shape = [1])
#     m3 = tf.get_variable('m3',dtype=tf.float32, shape = [1])
#     m1 = tf.sigmoid(m1)
#     m2 = tf.sigmoid(m2)
#     m3 = tf.sigmoid(m3)
    # init loss 
    loss = 0.0  
    # Mask 
    regularizer = tf.contrib.layers.l1_regularizer(scale=0.0005)
    reg_term    = tf.contrib.layers.apply_regularization(regularizer,[weights['att_wk']])
    # Start creat graph
    X  = tf.reshape(x,(1,n_frames,n_input))
    X2 = tf.reshape(x2,(1,n_frames,n_input))
    X3 = tf.reshape(x3,(1,n_frames,n_input))
    
    for i in range (n_frames):
        Xa  = X[0,i,:]
        X2a = X2[0,i,:]
        X3a = X3[0,i,:]
        Xa  = tf.spectral.fft( tf.cast(Xa,tf.complex64))
        X2a  = tf.spectral.fft( tf.cast(X2a,tf.complex64))
        X3a  = tf.spectral.fft( tf.cast(X3a,tf.complex64))
        Xa = tf.cast(Xa,tf.float32)
        X2a = tf.cast(X2a,tf.float32)
        X3a = tf.cast(X3a,tf.float32)
        
        if i == 0:
            Xb = tf.reshape(Xa,(1,1,n_hidden))
            X2b = tf.reshape(X2a,(1,1,n_hidden))
            X3b = tf.reshape(X3a,(1,1,n_hidden))
        else:
            temp_xb = tf.reshape(Xa,(1,1,n_hidden))
            temp_x2b = tf.reshape(X2a,(1,1,n_hidden))
            temp_x3b = tf.reshape(X3a,(1,1,n_hidden))
            Xb = tf.concat([Xb,temp_xb],axis = 1)
            X2b = tf.concat([X2b,temp_xb],axis = 1)
            X3b = tf.concat([X3b,temp_xb],axis = 1)
            
    # 1 rank approximation of attention weighting  
    wkeys1 = weights['att_wk']
    wkeys2 = weights['att_wk2']
    wkeys3 = weights['att_wk3']
        
        
    # Multi attention   
    multi_att, matt =  multihead_attention(queries=X, keys=Xb, wkeys = wkeys1, num_units=n_input, 
                                                        num_heads=8, 
                                                        dropout_rate=0.9,
                                                        is_training=1,
                                                        causality=False, 
                                                        scope="self_attention_1")
    multi_att2, _ =  multihead_attention(queries=X2, keys=X2b, wkeys = wkeys2, num_units=n_input, 
                                                        num_heads=8, 
                                                        dropout_rate=0.9,
                                                        is_training=1,
                                                        causality=False, 
                                                        scope="self_attention_2")
    multi_att3, _ =  multihead_attention(queries=X3, keys=X3b, wkeys = wkeys3, num_units=n_input, 
                                                        num_heads=8, 
                                                        dropout_rate=0.9,
                                                        is_training=1,
                                                        causality=False, 
                                                        scope="self_attention_3")
    
    multi_res = multi_att + multi_att2 + multi_att3 
    
    with tf.variable_scope("fc_1") as fc_1:
            fc_1 = tf.contrib.layers.fully_connected(tf.reshape(multi_res,(n_frames,1*n_hidden)), 3, activation_fn=tf.nn.relu)
            fc_1_drop = tf.nn.dropout(fc_1, 0.8)
            fc_1_variables =  [v for v in tf.global_variables() if v.name.startswith(fc_1.name)]        
    
    for i in range(n_frames):                  
        with tf.variable_scope('model',reuse=tf.AUTO_REUSE):
            
            mul1 =  tf.reshape(multi_att[0,i,:],(1,1*n_hidden))
            mul2 =  tf.reshape(multi_att2[0,i,:],(1,1*n_hidden))
            mul3 =  tf.reshape(multi_att3[0,i,:],(1,1*n_hidden))
            m1   =  fc_1_drop[i,0]
            m2   =  fc_1_drop[i,1]
            m3   =  fc_1_drop[i,2]
            m1   =  tf.sigmoid(m1)
            m2   =  tf.sigmoid(m2)
            m3   =  tf.sigmoid(m3)
            multi_ahead = tf.multiply(m1,mul1) + tf.multiply(m2,mul2) + tf.multiply(m3, mul3)
            
#             multi_ahead = tf.reshape(multi_res[0,i,:],(1,n_hidden))
            with tf.variable_scope("fc_m") as fc_m:
                fc_m = tf.contrib.layers.fully_connected(multi_ahead,n_hidden, activation_fn=tf.nn.relu)
                fc_m_drop = tf.nn.dropout(fc_m, 0.8)
                fc_m_variables =  [v for v in tf.global_variables() if v.name.startswith(fc_m.name)]   
            
            with tf.variable_scope("fc_m2") as fc_m2:
                fc_m2 = tf.contrib.layers.fully_connected(fc_m_drop,n_hidden, activation_fn=tf.nn.relu)
                fc_m2_drop = tf.nn.dropout(fc_m2, 0.8)
                fc_m_variables =  [v for v in tf.global_variables() if v.name.startswith(fc_m.name)]

            # FC to output classification
            pred = tf.matmul(fc_m2_drop , weights['out']) + biases['out']

            # save the predict of each time step
            if i == 0:
                soft_pred = tf.reshape((tf.nn.softmax(pred)),(1,n_classes))
 
            else:
                temp_soft_pred = tf.reshape((tf.nn.softmax(pred)),(1,n_classes))
                soft_pred   = tf.concat([soft_pred,temp_soft_pred],axis =0)



            # negative example
            yc = tf.reshape(y[0,:],(1,n_classes))
            neg_loss = tf.nn.softmax_cross_entropy_with_logits(labels = yc,logits = pred) # Softmax loss
            temp_loss = tf.reduce_mean(neg_loss)
            loss = tf.add(loss, temp_loss)
            #loss = tf.add(loss,reg_term)

      
    # Define loss and optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss/n_frames) # Adam Optimizer

    return x,x2,x3,y,optimizer,loss,soft_pred,fc_m_variables,multi_att,matt

In [8]:
tf.reset_default_graph()
x,x2,x3,y,optimizer,loss,soft_pred,fc_m_variables,multi_att,matt= build_model()
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.8)
sess = tf.InteractiveSession(config=tf.ConfigProto(allow_soft_placement=True,gpu_options=gpu_options))
# mkdir folder for saving model
if os.path.isdir(save_path) == False:
        os.mkdir(save_path)
# Initializing the variables
init = tf.global_variables_initializer()
# Launch the graph
sess.run(init)
saver = tf.train.Saver()
#saver = tf.train.import_meta_graph(save_path +'model-25.meta')
#saver.restore(sess,save_path+'model-25')
epoch = 1
epoch_loss = np.zeros((train_num,1),dtype = float)
n_batchs = np.arange(1,train_num+1)
np.random.shuffle(n_batchs)
tStart_epoch = time.time()
matt.get_shape

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



<bound method Tensor.get_shape of <tf.Tensor 'self_attention_1/Reshape_4:0' shape=(8, 64, 64) dtype=float32>>

In [9]:
# Keep training until reach max iterations
# start training
for epoch in range(n_epochs):
#  chose batch randomly
    epoch_loss = np.zeros((train_num,1),dtype = float)
    n_batchs = np.arange(1,train_num+1)
    np.random.shuffle(n_batchs)
    tStart_epoch = time.time()
    k = 0
    for batch in n_batchs:
        
        # load features and labels
        file_name    = video_list['video'][0][batch][0][0]
        batch_data   = np.load(train_path   + file_name +'.npy').item()
        batch_data2  = np.load(train_path_2 + file_name +'.npy').item()
        batch_data3  = np.load(train_path_3 + file_name +'.npy').item()
        batch_xs     = batch_data['feat']
        batch_xs2    = batch_data2['feat']
        batch_xs3    = batch_data3['feat']
        
        batch_ys     = batch_data['label']
        totalframe   = batch_xs.shape
        totalframe   = totalframe[0]
        
        # reshape input batch 
        batch_xc  = np.zeros((1,n_frames,batch_xs.shape[1]))
        batch_xc2 = np.zeros((1,n_frames,batch_xs.shape[1]))
        batch_xc3 = np.zeros((1,n_frames,batch_xs.shape[1]))
        batch_yc  = np.zeros((1,n_frames,n_classes))                   
        batch_xc[0,:,:]  = batch_xs
        batch_xc2[0,:,:] = batch_xs2
        batch_xc3[0,:,:] = batch_xs3
        
        # feed-forward
        _,batch_loss = sess.run([optimizer,loss], feed_dict={x: batch_xc, x2: batch_xc2, x3: batch_xc3, y: batch_ys})

        epoch_loss[batch-1] = batch_loss
        # Debug per batch
        k = k + 1

    # print per epoch
    print ("Epoch:", epoch+1, " done. Loss:", np.mean(epoch_loss))
    tStop_epoch = time.time()
    print ("Epoch Time Cost:", round(tStop_epoch - tStart_epoch,2), "s")
    sys.stdout.flush()
    if (epoch+1) %10 == 0:
        saver.save(sess,save_path+"model", global_step = epoch+1)
print ("Optimization Finished!")
saver.save(sess, save_path+"final_model")

('Epoch:', 1, ' done. Loss:', 167.2511212670125)
('Epoch Time Cost:', 573.43, 's')
('Epoch:', 2, ' done. Loss:', 64.87989568048981)
('Epoch Time Cost:', 581.16, 's')
('Epoch:', 3, ' done. Loss:', 37.59624565376241)
('Epoch Time Cost:', 583.81, 's')
('Epoch:', 4, ' done. Loss:', 23.200200692858807)
('Epoch Time Cost:', 626.89, 's')
('Epoch:', 5, ' done. Loss:', 14.561231249255517)
('Epoch Time Cost:', 730.24, 's')
('Epoch:', 6, ' done. Loss:', 8.676740928826518)
('Epoch Time Cost:', 730.53, 's')
('Epoch:', 7, ' done. Loss:', 5.09800666674873)
('Epoch Time Cost:', 813.72, 's')
('Epoch:', 8, ' done. Loss:', 2.9456533519917465)
('Epoch Time Cost:', 729.35, 's')
('Epoch:', 9, ' done. Loss:', 1.781063133785946)
('Epoch Time Cost:', 1039.21, 's')
('Epoch:', 10, ' done. Loss:', 1.1203075809394452)
('Epoch Time Cost:', 1089.3, 's')
('Epoch:', 11, ' done. Loss:', 0.7331767681936201)
('Epoch Time Cost:', 716.45, 's')
('Epoch:', 12, ' done. Loss:', 0.4209614632706482)
('Epoch Time Cost:', 777.9, '

('Epoch:', 97, ' done. Loss:', 0.0016507645413688064)
('Epoch Time Cost:', 415.23, 's')
('Epoch:', 98, ' done. Loss:', 0.000947154102801278)
('Epoch Time Cost:', 416.17, 's')
('Epoch:', 99, ' done. Loss:', 0.060342707472499015)
('Epoch Time Cost:', 416.5, 's')
('Epoch:', 100, ' done. Loss:', 0.0009365917068465942)
('Epoch Time Cost:', 416.72, 's')
('Epoch:', 101, ' done. Loss:', 0.14251318218057338)
('Epoch Time Cost:', 415.74, 's')
('Epoch:', 102, ' done. Loss:', 0.0008176019470214552)
('Epoch Time Cost:', 415.43, 's')
('Epoch:', 103, ' done. Loss:', 0.00019528876342324325)
('Epoch Time Cost:', 416.34, 's')
('Epoch:', 104, ' done. Loss:', 0.017365195124229935)
('Epoch Time Cost:', 415.66, 's')
('Epoch:', 105, ' done. Loss:', 0.011923851403162954)
('Epoch Time Cost:', 415.25, 's')
('Epoch:', 106, ' done. Loss:', 0.00048655831953695907)
('Epoch Time Cost:', 416.88, 's')
('Epoch:', 107, ' done. Loss:', 0.011030494431518661)
('Epoch Time Cost:', 416.83, 's')
('Epoch:', 108, ' done. Loss:'

'./multi_model/multi_att_1_fusionft_3s/final_model'

In [10]:
tf.reset_default_graph()
att_res = {}
x,x2,x3,y,optimizer,loss,soft_pred,fc_m_variables,multi_att,matt= build_model()
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.5)
sess = tf.InteractiveSession(config=tf.ConfigProto(allow_soft_placement=True,gpu_options=gpu_options))
init = tf.global_variables_initializer()
sess.run(init)

saver = tf.train.Saver()

# restore model
print(default_model_path)
saver.restore(sess, default_model_path)
top1_list = []
for num_batch in range(0,test_num+1): # 0, test_num = 0 : test_num-1
    file_name = test_list['video'][0][num_batch][0][0]
    test_all_data = np.load(test_path + file_name+'.npy').item()
    test_all_data2 = np.load(test_path_2 + file_name+'.npy').item()
    test_all_data3 = np.load(test_path_3 + file_name+'.npy').item()
    print(file_name)
    test_data = test_all_data['feat']
    test_data2 = test_all_data2['feat']
    test_data3 = test_all_data3['feat']
   
    #test_data_final = np.concatenate ((test_data,test_data_2),axis = 2)
    test_labels = test_all_data['label']
        
    totalframe = test_data.shape
    totalframe = totalframe[0]
    att_res['name'] = file_name
    #print(totalframe)
    pred_list = np.zeros((n_frames,n_classes))
    batch_xc  = np.zeros((1,n_frames,test_data.shape[1]))
    batch_xc2 = np.zeros((1,n_frames,test_data.shape[1]))
    batch_xc3 = np.zeros((1,n_frames,test_data.shape[1]))
    batch_xc[0,:,:] = test_data
    batch_xc2[0,:,:] = test_data2
    batch_xc3[0,:,:] = test_data3
                 
    [pred,matte] = sess.run([soft_pred,matt], feed_dict={x: batch_xc, x2: batch_xc2, x3: batch_xc3, y:test_labels})
   
    pred_list    = pred
    matt_list    = matte
    

    label_index  = np.where(test_labels==1)[1][0]
    
    avg_pre = np.mean(pred_list, axis=0).tolist()
    top1 = (avg_pre.index(max(avg_pre))==label_index)
    top1_list.append(top1)
    att_res['pred_list'] = avg_pre
    att_res['label'] = test_labels 
    att_res['matt'] = matt_list
  
    print('video number:' ,num_batch, ' of ', test_num)
    print('current mAP = ',np.mean(top1_list))
    
    sio.savemat(output_path+file_name+'.mat',att_res, do_compression=False)

/usr/local/lib/python2.7/dist-packages/tensorflow/python/client/session.py:1645: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


./multi_model/multi_att_1_fusionft_3s/final_model
INFO:tensorflow:Restoring parameters from ./multi_model/multi_att_1_fusionft_3s/final_model
Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_ba_med_3
('video number:', 0, ' of ', 1529)
('current mAP = ', 1.0)
Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_fr_goo_1
('video number:', 1, ' of ', 1529)
('current mAP = ', 1.0)
Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_fr_goo_2
('video number:', 2, ' of ', 1529)
('current mAP = ', 1.0)
Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_fr_med_0
('video number:', 3, ' of ', 1529)
('current mAP = ', 1.0)
Brushing_Her_Hair__[_NEW_AUDIO_]_UPDATED_brush_hair_h_cm_np1_fr_goo_0
('video number:', 4, ' of ', 1529)
('current mAP = ', 1.0)
Brushing_Her_Hair__[_NEW_AUDIO_]_UPDATED_brush_hair_h_cm_np1_le_goo_1
('video number:', 5, ' of ', 1529)
('current mAP = ', 1.0)
Brushing_Her_Hair__[_NEW_AUDIO_]_UPDATED_brush_hair_h_cm_np1_le_goo_2
('video number:', 6, ' of ', 1529)
('c

('video number:', 62, ' of ', 1529)
('current mAP = ', 0.6825396825396826)
Florian_Fromlowitz_beim_Training_der_U_21_Nationalmannschaft_catch_f_cm_np1_ri_med_4
('video number:', 63, ' of ', 1529)
('current mAP = ', 0.671875)
Florian_Fromlowitz_beim_Training_der_U_21_Nationalmannschaft_catch_u_cm_np1_le_med_1
('video number:', 64, ' of ', 1529)
('current mAP = ', 0.676923076923077)
Florian_Fromlowitz_beim_Training_der_U_21_Nationalmannschaft_catch_u_cm_np1_le_med_3
('video number:', 65, ' of ', 1529)
('current mAP = ', 0.6666666666666666)
Florian_Fromlowitz_beim_Training_der_U_21_Nationalmannschaft_catch_u_cm_np1_le_med_5
('video number:', 66, ' of ', 1529)
('current mAP = ', 0.6716417910447762)
Goalkeeper_Training_Day__2_catch_f_cm_np1_ba_bad_3
('video number:', 67, ' of ', 1529)
('current mAP = ', 0.6617647058823529)
Goalkeeper_Training_Day__2_catch_f_cm_np1_ba_bad_4
('video number:', 68, ' of ', 1529)
('current mAP = ', 0.6521739130434783)
Goalkeeper_Training_Day__2_catch_f_cm_np1_ba

('video number:', 126, ' of ', 1529)
('current mAP = ', 0.6141732283464567)
LetThePeopleClap_clap_u_cm_np1_fr_med_2
('video number:', 127, ' of ', 1529)
('current mAP = ', 0.6171875)
Martin_klatscht_in_die_h_nde_und_FURZT_clap_u_cm_np1_fr_med_0
('video number:', 128, ' of ', 1529)
('current mAP = ', 0.6124031007751938)
Return_of_the_King_12_clap_u_nm_np1_fr_goo_4
('video number:', 129, ' of ', 1529)
('current mAP = ', 0.6153846153846154)
Standing_clap_f_cm_np1_fr_med_0
('video number:', 130, ' of ', 1529)
('current mAP = ', 0.6106870229007634)
Standing_clap_f_cm_np1_fr_med_1
('video number:', 131, ' of ', 1529)
('current mAP = ', 0.6136363636363636)
The_Slow_Clap_clap_u_cm_np1_fr_med_3
('video number:', 132, ' of ', 1529)
('current mAP = ', 0.6165413533834586)
The_Slow_Clap_clap_u_cm_np1_fr_med_4
('video number:', 133, ' of ', 1529)
('current mAP = ', 0.6119402985074627)
The_Slow_Clap_clap_u_cm_np2_fr_med_17
('video number:', 134, ' of ', 1529)
('current mAP = ', 0.6148148148148148)
Th

('video number:', 186, ' of ', 1529)
('current mAP = ', 0.5828877005347594)
India_Monkey_King_scales_new_heights_climb_f_cm_np1_ba_med_4
('video number:', 187, ' of ', 1529)
('current mAP = ', 0.5851063829787234)
Kletter-WM_2005-_Bouldern_Finale_Damen__Vera_Kotasova_CZE_climb_f_cm_np1_ba_med_0
('video number:', 188, ' of ', 1529)
('current mAP = ', 0.5873015873015873)
Kletter-WM_2005-_Bouldern_Finale_Damen__Vera_Kotasova_CZE_climb_f_cm_np1_ba_med_1
('video number:', 189, ' of ', 1529)
('current mAP = ', 0.5894736842105263)
Kletter-WM_2005-_Bouldern_Finale_Damen__Vera_Kotasova_CZE_climb_f_cm_np1_ba_med_2
('video number:', 190, ' of ', 1529)
('current mAP = ', 0.5916230366492147)
Kletter-WM_2005-_Bouldern_Finale_Herren__Serik_Kazbekov_climb_f_cm_np1_ba_med_0
('video number:', 191, ' of ', 1529)
('current mAP = ', 0.59375)
Kletter-WM_2005-_Bouldern_Finale_Herren__Serik_Kazbekov_climb_f_cm_np1_ba_med_1
('video number:', 192, ' of ', 1529)
('current mAP = ', 0.5958549222797928)
Kletter-WM_2

...

('video number:', 1224, ' of ', 1529)
('current mAP = ', 0.5477551020408163)
smoking_a_cigarette_smoke_h_nm_np1_fr_med_1
('video number:', 1225, ' of ', 1529)
('current mAP = ', 0.5473083197389886)
smoking_a_cigarette_smoke_h_nm_np1_fr_med_2
('video number:', 1226, ' of ', 1529)
('current mAP = ', 0.5476772616136919)
zigarette_rauchen_smoke_h_nm_np1_fr_med_0
('video number:', 1227, ' of ', 1529)
('current mAP = ', 0.5480456026058632)
zigarette_rauchen_smoke_h_nm_np1_fr_med_1
('video number:', 1228, ' of ', 1529)
('current mAP = ', 0.548413344182262)
zigarette_rauchen_smoke_h_nm_np1_fr_med_2
('video number:', 1229, ' of ', 1529)
('current mAP = ', 0.5487804878048781)
Bodenk_r_Spo_08_somersault_f_cm_np1_le_med_0
('video number:', 1230, ' of ', 1529)
('current mAP = ', 0.5491470349309504)
Kristina_Goryunova_-_Floor_Exercise_-_2009_Tyson_American_Cu_somersault_f_cm_np1_le_med_1
('video number:', 1231, ' of ', 1529)
('current mAP = ', 0.549512987012987)
Monikas_Staatsexamen__Turnen_somersau

('video number:', 1290, ' of ', 1529)
('current mAP = ', 0.5491866769945778)
BaseBallSweetSwingsingle_swing_baseball_f_nm_np1_ba_bad_0
('video number:', 1291, ' of ', 1529)
('current mAP = ', 0.5495356037151703)
Faith_Rewarded_swing_baseball_f_cm_np1_ba_bad_11
('video number:', 1292, ' of ', 1529)
('current mAP = ', 0.5491105955143079)
Faith_Rewarded_swing_baseball_f_cm_np1_le_bad_21
('video number:', 1293, ' of ', 1529)
('current mAP = ', 0.5486862442040186)
Faith_Rewarded_swing_baseball_f_cm_np1_ri_bad_37
('video number:', 1294, ' of ', 1529)
('current mAP = ', 0.5482625482625483)
Faith_Rewarded_swing_baseball_f_nm_np0_fr_bad_67
('video number:', 1295, ' of ', 1529)
('current mAP = ', 0.5478395061728395)
Faith_Rewarded_swing_baseball_f_nm_np1_ba_bad_22
('video number:', 1296, ' of ', 1529)
('current mAP = ', 0.5474171164225135)
Faith_Rewarded_swing_baseball_f_nm_np1_ba_bad_77
('video number:', 1297, ' of ', 1529)
('current mAP = ', 0.5469953775038521)
Faith_Rewarded_swing_baseball_f_

('video number:', 1353, ' of ', 1529)
('current mAP = ', 0.5324963072378139)
HIGHLANDER___TRAINING___WEEPING_OF_THE_SPIRITS_sword_u_cm_np2_fr_med_1
('video number:', 1354, ' of ', 1529)
('current mAP = ', 0.5321033210332103)
Medieval_Times_Knights_Battle_Sword_Fight_sword_f_cm_np2_ba_med_6
('video number:', 1355, ' of ', 1529)
('current mAP = ', 0.5317109144542773)
Medieval_Times_Knights_Battle_Sword_Fight_sword_f_cm_np2_ba_med_7
('video number:', 1356, ' of ', 1529)
('current mAP = ', 0.5313190862196021)
Medieval_Times_Knights_Battle_Sword_Fight_sword_f_cm_np2_fr_med_5
('video number:', 1357, ' of ', 1529)
('current mAP = ', 0.5309278350515464)
Medieval_Times_Knights_Battle_Sword_Fight_sword_f_cm_np2_le_bad_0
('video number:', 1358, ' of ', 1529)
('current mAP = ', 0.5305371596762325)
Medieval_Times_Knights_Battle_Sword_Fight_sword_f_cm_np2_le_med_3
('video number:', 1359, ' of ', 1529)
('current mAP = ', 0.5301470588235294)
Medieval_Times_Knights_Battle_Sword_Fight_sword_f_cm_np2_ri_

('video number:', 1419, ' of ', 1529)
('current mAP = ', 0.5267605633802817)
KnifeThrowingJackDaggerDemoReel_throw_u_nm_np1_ba_med_3
('video number:', 1420, ' of ', 1529)
('current mAP = ', 0.5263898662913441)
KnifeThrowingJackDaggerDemoReel_throw_u_nm_np1_fr_med_5
('video number:', 1421, ' of ', 1529)
('current mAP = ', 0.5260196905766527)
KnifeThrowing_throw_f_nm_np1_le_med_0
('video number:', 1422, ' of ', 1529)
('current mAP = ', 0.5256500351370345)
KnifeThrowing_throw_f_nm_np1_le_med_1
('video number:', 1423, ' of ', 1529)
('current mAP = ', 0.5252808988764045)
KnifeThrowing_throw_f_nm_np1_le_med_2
('video number:', 1424, ' of ', 1529)
('current mAP = ', 0.5249122807017544)
KnifeThrowing_throw_f_nm_np1_le_med_3
('video number:', 1425, ' of ', 1529)
('current mAP = ', 0.5245441795231417)
KnifeThrowing_throw_f_nm_np1_le_med_4
('video number:', 1426, ' of ', 1529)
('current mAP = ', 0.524176594253679)
Pirates_4_throw_u_nm_np1_ba_med_4
('video number:', 1427, ' of ', 1529)
('current m

Prelinger_LetYours1940_walk_f_nm_np1_ba_med_5
('video number:', 1489, ' of ', 1529)
('current mAP = ', 0.5234899328859061)
RATRACE_walk_f_cm_np1_ba_med_17
('video number:', 1490, ' of ', 1529)
('current mAP = ', 0.5231388329979879)
SafeInPort_walk_f_nm_np1_ba_med_2
('video number:', 1491, ' of ', 1529)
('current mAP = ', 0.5234584450402144)
Sixthsense_walk_u_nm_np1_fr_med_4
('video number:', 1492, ' of ', 1529)
('current mAP = ', 0.5231078365706631)
Veoh_Alpha_Dog_1_walk_f_nm_np1_ri_med_24
('video number:', 1493, ' of ', 1529)
('current mAP = ', 0.5227576974564926)
Veoh_Alpha_Dog_1_walk_u_cm_np1_fr_med_13
('video number:', 1494, ' of ', 1529)
('current mAP = ', 0.5230769230769231)
Veoh_Alpha_Dog_2_walk_f_cm_np2_ri_med_10
('video number:', 1495, ' of ', 1529)
('current mAP = ', 0.5227272727272727)
Veoh_Alpha_Dog_2_walk_f_nm_np4_fr_med_17
('video number:', 1496, ' of ', 1529)
('current mAP = ', 0.5230460921843687)
Veoh_Alpha_Dog_2_walk_u_nm_np1_le_med_29
('video number:', 1497, ' of ', 1

In [11]:
test_labels

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1.]])

In [12]:
np.where(test_labels==1)[1][0]

50

In [13]:
sum(top1_list)

787